## EfficientNetB0 Model

### Import the libraries

In [1]:
!pip install tensorflow_hub

In [2]:
pip install tf_keras

In [3]:
!pip install tensorflow numpy matplotlib opencv-python

In [4]:
#import the libraries
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

### Load efficientnetb0 base model

In [5]:
base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base.trainable= False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Create the parameters for custom model

In [6]:
x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.4)(x)
x = Dense(128, activation = "relu")(x)
x = Dropout(0.2)(x)
outputlayer = Dense(1, activation = "sigmoid")


### Create the model

In [7]:
detector = Model(inputs=base.input, outputs=outputlayer(x))

#compile
detector.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

detector.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

### Set training parameters

In [10]:
#rescale and normalize data
datagen_train = ImageDataGenerator(rescale = 1.0/255, rotation_range = 30, width_shift_range = 0.2, height_shift_range = 0.2, zoom_range = 0.2, horizontal_flip = True, validation_split= 0.2)

### set up train and test data parameters


In [11]:
#unzip file
import zipfile

with zipfile.ZipFile("/content/imagedataset.zip", "r") as dataset:
  dataset.extractall("/content/imagedataset/")

train_detector = datagen_train.flow_from_directory("/content/imagedataset/imagedataset/dataset/", target_size=(224, 224), batch_size=32, class_mode = "binary", subset = "training")

validation_detector = datagen_train.flow_from_directory("/content/imagedataset/imagedataset/dataset/", target_size=(224, 224), batch_size=32, class_mode = "binary", subset = "validation")

Found 861 images belonging to 2 classes.
Found 214 images belonging to 2 classes.


###Train the model

In [12]:
history = detector.fit(train_detector, epochs=50, validation_data = validation_detector)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
19/27 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.4858 - loss: 0.7335

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


27/27 ━━━━━━━━━━━━━━━━━━━━ 133s 4s/step - accuracy: 0.4852 - loss: 0.7315 - val_accuracy: 0.5000 - val_loss: 0.6968
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - accuracy: 0.4856 - loss: 0.6957 - val_accuracy: 0.5000 - val_loss: 0.6948
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 112s 4s/step - accuracy: 0.4518 - loss: 0.7013 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 112s 4s/step - accuracy: 0.5248 - loss: 0.6919 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.4983 - loss: 0.6971 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step - accuracy: 0.5062 - loss: 0.6944 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.5383 - loss: 0.6928 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 123s 4s/step - accuracy: 0.5118 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.

### Fine tune the model

In [15]:
base.trainable = True

detector.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss = 'binary_crossentropy', metrics=['accuracy'])

history_finetune= detector.fit(train_detector, epochs=5, validation_data = validation_detector)

Epoch 1/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 420s 13s/step - accuracy: 0.4727 - loss: 0.8143 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 307s 11s/step - accuracy: 0.5573 - loss: 0.7507 - val_accuracy: 0.4766 - val_loss: 0.6931
Epoch 3/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 307s 11s/step - accuracy: 0.5689 - loss: 0.7125 - val_accuracy: 0.4766 - val_loss: 0.6933
Epoch 4/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 308s 11s/step - accuracy: 0.5677 - loss: 0.6944 - val_accuracy: 0.4159 - val_loss: 0.6937
Epoch 5/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 303s 11s/step - accuracy: 0.6220 - loss: 0.6592 - val_accuracy: 0.4299 - val_loss: 0.6938


### model evaluation

In [18]:
val_loss, val_accuracy = detector.evaluate(validation_detector)
print(f"Validation Accuracy: {val_accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.4558 - loss: 0.6935
Validation Accuracy: 0.43


### Test on new images

In [ ]:
import cv2

In [ ]:
def predict_image(image_path):
  img= cv2.imread(image_path)
  img = cv2.resize(img,(224, 224)) / 255.0
  img= np.expand_dims(img, axis=0)
  prediction = detector.predict(img)
  return "Class 1 (Contain AI generation)" if prediction > 0.5 else "Class 0 (Little to No AI generation detected)"

  image_path= "Bill_Clinton.jpg"
  result = predict_image(image_path)
  print(result)


### Calsulate the accuracy

In [ ]:
#f1 score
#aucroc
#confusion matrix
#precision score
#recall score

## VGG16 Model

In [17]:
# train_data_vgg= ImageDataGenerator()
# traindatavgg = train_data_vgg.flow_from_directory(directory = dataset, target_size=(224, 224))

# test_data_vgg = ImageDataGenerator()
# testdatavgg = test_data_vgg.flow_from_directory(directory = dataset, target_size=(224, 224))

FileNotFoundError: [Errno 2] No such file or directory: 'dataset'

### create an object for training and testing data

In [25]:
!pip install keras

In [34]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

detector_vgg = Sequential()
detector_vgg.add(Conv2D(input_shape=(22, 224,3), filters=64, kernel_size=(3,3), padding="same", activation="relu"))
detector_vgg.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation = "relu"))
detector_vgg.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
detector_vgg.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
detector_vgg.add(Conv2D(filters = 128, kernel_size=(3,3), padding="same", activation="relu"))
detector_vgg.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
detector_vgg.add(Conv2D(filters=256, kernel_size=(3,3), padding = "same", activation = "relu"))
detector_vgg.add(Conv2D(filters=256, kernel_size=(3, 3), padding = "same", activation ="relu"))
detector_vgg.add(Conv2D(filters=256, kernel_size=(3,3), padding = "same", activation="relu"))
detector_vgg.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
detector_vgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
detector_vgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation ="relu"))
detector_vgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation = "relu"))
detector_vgg.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
detector_vgg.add(Conv2D(filters=512, kernel_size=(3,3), padding= "same", activation = "relu"))
detector_vgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation = "relu"))
detector_vgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation = "relu"))
detector_vgg.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


#initialize the model
detector_vgg.add(Flatten())
detector_vgg.add(Dense(units=4096, activation="relu"))
detector_vgg.add(Dense(units=4096, activation="relu"))
detector_vgg.add(Dense(units=2, activation="softmax"))

#### compile model

In [ ]:
from keras.optimizers import Adam

optim = Adam(lr=0.001)
detector_vgg

In [ ]:
#unzip file
import zipfile

with zipfile.ZipFile("/content/imagedataset.zip", "r") as dataset_vgg:
  dataset.extractall("/content/imagedataset/")

train_detector_vgg= datagen_train.flow_from_directory("/content/imagedataset/imagedataset/dataset/", target_size=(224, 224), batch_size=32, class_mode = "binary", subset = "training")

validation_detector_vgg = datagen_train.flow_from_directory("/content/imagedataset/imagedataset/dataset/", target_size=(224, 224), batch_size=32, class_mode = "binary", subset = "validation")

In [ ]:
# #import the libraries
# import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import matplotlib.pyplot as plt
# import numpy as np
# import os

# base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
# base.trainable= False

In [ ]:
# #install required libraries
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.image import imread
# import tensorflow_hub as hub
# import tf_keras
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras import layers, models
# import keras
# from keras.models import Sequential
# from keras.layers import InputLayer

# #load efficientnet
# base=hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/classification/1")
# base.trainable = False

In [ ]:
# #sequential model
# model= tf_keras.Sequential()
# #add efficientnet
# model.add(base)
# #add global average pooling layer
# model.add(layers.GlobalAveragePooling2D())
# #add a dense layer for classification
# model.add(layers.Dense(num_classes=2, activation="softmax"))

In [ ]:
# from matplotlib import pyplot
# import matplotlib.pyplot as plt
# from matplotlib.image import imread
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Conv2D
# from keras.layers import MaxPooling2D
# from keras.layers import Dense, Dropout, Flatten
# # from keras.optimizers import SGD
# from keras.models import Model
# from keras.src.legacy.preprocessing.image import ImageDataGenerator

# folder = 'dataset/'

In [ ]:
# def displayImages(foldername, aiorreal, startID):
#   # for i in range(9):
#     pyplot.subplot(330 + 1 + i)
#     filename